## Generate Synthetic Samples

In [ ]:
import os
from TFR_simu import simu_cat_TFR

""" TFR parameters """
beta  = 3.33
gamma = 10.5
""" Schechter function """
M_star = 10.8
alpha  = -1.27
v_star = M_star-gamma  # bv* for velocity function
""" sensitivity limit """
ml = 5.736
indir = 'data_step'
#ml = [5.736,0.128]
#indir = 'data_erf'
if not os.path.exists(indir): os.mkdir(indir)

""" simulate samples """
czmin,czmax,czbin = 3000, 19000, 50
for simu in ['A','B','C']:
    if simu == 'A': sigm,sigw = 0.15,   0.0
    if simu == 'B': sigm,sigw =  0.0, 0.045
    if simu == 'C': sigm,sigw = 0.15, 0.045

    simfile = indir+'/Sample_'+simu+'.csv'
    if not os.path.exists(simfile):
        cat = simu_cat_TFR(czmin, czmax, czbin, nn=-1, oversample=2.2, 
            sigm=sigm, sigw=sigw, 
            beta=beta, gamma=gamma, v_star=v_star, alpha=alpha, ml=ml)            
        cat.to_csv(simfile)
    else:
        print(f'File already exist: {simfile=}')

## Parameter Inference with MCMC 

- While testing, use `nstep = 200` and `nrepeat = 1`, and disable convergence test `converge_check=False`. 
- For production run, use `nstep = 1000` and `nrepeat = 10` and enable convergence test `converge_check=True`.

In [ ]:
""" GPU- and FFT-accelerated Complete Model """
from TFR_mcmc_gpu import run_emcee_gpu
import pandas as pd
import numpy as np

czlmt = [4,18] # redshift limits in 1000 km/s
indir = 'data_step'
samples = ['Sample_C'] # 'Sample_A','Sample_B'
for sample in samples:
    # Load Input Data
    datafile = indir+'/'+sample+'.csv'
    cat = pd.read_csv(datafile)
    idx = (cat.Vcmb/1e3 > czlmt[0]) & (cat.Vcmb/1e3 <= czlmt[1]) 
    df = cat.loc[idx, :]
    ndata = (df.shape)[0]
    ws = np.array(df['logW']-2.5)
    ms = np.array(df['logmb']) 
    ds = np.array(df['d'])
    # Run MCMC sampler
    method = 'unifgpu'
    outdir = datafile.replace('.csv','_')+method
    _,_ = run_emcee_gpu(ws,ms,ds, outdir=outdir, 
                    ml=5.736, nsteps=1000, nrepeat=10, nwalkers=16,
                    converge_check=True)

In [ ]:
""" CPU-powered Forward, Inverse, and Complete Models """
from TFR_mcmc import run_emcee
import pandas as pd
import numpy as np

czlmt = [4,18] # redshift limits in 1000 km/s
indir = 'data_step'
samples = ['Sample_A','Sample_B'] #,'Sample_C']
for sample in samples:
    # Load Input Data
    datafile = indir+'/'+sample+'.csv'
    cat = pd.read_csv(datafile)
    idx = (cat.Vcmb/1e3 > czlmt[0]) & (cat.Vcmb/1e3 <= czlmt[1]) 
    df = cat.loc[idx, :]
    ndata = (df.shape)[0]
    ws = np.array(df['logW']-2.5)
    ms = np.array(df['logmb']) 
    ds = np.array(df['d'])
    # Run MCMC sampler
    for method in ['forward','inverse']: #,'unified']:
        outdir = datafile.replace('.csv','_')+method
        _,_ = run_emcee(ws,ms,ds, outdir=outdir, 
                         ml=5.736, nsteps=1000, nrepeat=10, ncpu=8,
                         converge_check=True)

## Make Posterior Corner Plots with ChainConsumer

In [ ]:
from chainconsumer import Chain, ChainConfig, ChainConsumer, Truth, PlotConfig
import numpy as np
import emcee, glob
import matplotlib.pyplot as plt

indir = 'data_step'
# max (default; mode + iso), cumulative (median), max_central (mode + cumu), or mean 
stat_method = 'cumulative'
samples = ['Sample_A','Sample_B','Sample_C']
for i in range(len(samples)):
    sample = samples[i]
    # start an empty consumer
    c = ChainConsumer()
    c.set_override(ChainConfig(statistics='cumulative'))
    
    # truth values of the synthetic samples
    if sample == 'Sample_A':
        truthloc = {"$\gamma$": 10.5, "$\\beta$": 3.33, 
                "$\sigma_m$": 0.15, "$\sigma_w$": 0.0,
                "$v_\star$": 0.30, "$\\alpha$": -1.27}
        slabel = '(A)'
    if sample == 'Sample_B':
        truthloc = {"$\gamma$": 10.5, "$\\beta$": 3.33, 
                "$\sigma_m$": 0.0, "$\sigma_w$": 0.045,
                "$v_\star$": 0.30, "$\\alpha$": -1.27}
        slabel = '(B)'
    if sample == 'Sample_C':
        truthloc = {"$\gamma$": 10.5, "$\\beta$": 3.33, 
                "$\sigma_m$": 0.15, "$\sigma_w$": 0.045,
                "$v_\star$": 0.30, "$\\alpha$": -1.27}
        slabel = '(C)'

    # convert emcee chains to ChainConsumer chains
    for method in ['forward','inverse','unified']:
        if method == 'forward':
            params = ["$\gamma$", "$\\beta$", "$\sigma_m$", "$v_\star$", "$\\alpha$"]
        elif method == 'inverse':
            params = ["$\gamma$", "$\\beta$", "$\\sigma_w$"]
        elif method == 'unified':
            params = ["$\gamma$", "$\\beta$", "$\sigma_m$", "$\sigma_w$", "$v_\star$", "$\\alpha$"]

        # search for emcee .h5 file
        emcfile = glob.glob(f'{indir}/{sample}_{method}/emcee_v4-18*.h5')
        if len(emcfile) == 0: continue
        sampler = emcee.backends.HDFBackend(emcfile[0])
        # estimate autocorrelation (converged if chain length > 50x tau)
        tau = sampler.get_autocorr_time(quiet=True)
        #print(emcfile,np.max(tau))
        burnin = int(2 * np.max(tau))
        thin = int(0.5 * np.min(tau))
        # convert to ChainConsumer format
        chain = Chain.from_emcee(sampler, params, method, discard=burnin, thin=thin)
        # add chain to ChainConsumer
        c.add_chain(chain)

    """ generate corner plots """
    c.add_truth(Truth(location=truthloc))
    c.add_marker(location=truthloc, name="Truth", color="black", marker_style="P", marker_size=50)

    col2plot = ["$\\beta$", "$\gamma$"]
    extents = {"$\gamma$": (10.42, 10.58), "$\\beta$": (2.97, 3.71)}
    c.set_plot_config(PlotConfig(extents=extents,spacing=0,serif=True,flip=True,legend_kwargs={'loc':'lower right'}))
    fig = c.plotter.plot(columns=col2plot,figsize=0.5)
    fig.axes[2].text(3.02,10.56,slabel,fontsize=12)
    plt.savefig(f'{indir}/{sample}_c{len(col2plot)}.pdf', bbox_inches='tight')

    if sample == 'Sample_C':
        col2plot = ["$\\beta$", "$\gamma$", "$v_\star$", "$\\alpha$", "$\sigma_w$", "$\sigma_m$"]
        c.set_plot_config(PlotConfig(legend_location=(len(col2plot)-2,len(col2plot)-1)))
        fig = c.plotter.plot(columns=col2plot,figsize='PAGE')
        plt.savefig(f'{indir}/{sample}_c{len(col2plot)}.pdf', bbox_inches='tight');
